In [0]:
!wget https://raw.githubusercontent.com/chen0040/keras-sentiment-analysis-web-api/master/demo/data/umich-sentiment-train.txt -P ./data

In [1]:
# -*- coding: utf-8 -*-
from __future__ import division, print_function
import collections
import os

import nltk
import numpy as np
from keras.callbacks import TensorBoard
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import codecs

Using TensorFlow backend.


In [0]:
DATA_DIR = "./data"
LOG_DIR = "./logs"

MAX_FEATURES = 2000
MAX_SENTENCE_LENGTH = 40

EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10

In [3]:
# Read training data and generate vocabulary
maxlen = 0
word_freqs = collections.Counter()
num_recs = 0
with codecs.open(os.path.join(DATA_DIR, "umich-sentiment-train.txt"), "r",
                 'utf-8') as ftrain:
    for line in ftrain:
        label, sentence = line.strip().split("\t")
        try:
            words = nltk.word_tokenize(sentence.lower())
        except LookupError:
            print("Englisth tokenize does not downloaded. So download it.")
            nltk.download("punkt")
            words = nltk.word_tokenize(sentence.lower())
        maxlen = max(maxlen, len(words))
        for word in words:
            word_freqs[word] += 1
        num_recs += 1

# Get some information about our corpus
print(maxlen)            # 42
print(len(word_freqs))   # 2313

# 1 is UNK, 0 is PAD
# We take MAX_FEATURES-1 features to account for PAD
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
word2index = {x[0]: i+2 for i, x in
              enumerate(word_freqs.most_common(MAX_FEATURES))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v: k for k, v in word2index.items()}

# convert sentences to sequences
X = np.empty((num_recs, ), dtype=list)
y = np.zeros((num_recs, ))
i = 0
with codecs.open(os.path.join(DATA_DIR, "umich-sentiment-train.txt"),
                 'r', 'utf-8') as ftrain:
    for line in ftrain:
        label, sentence = line.strip().split("\t")
        words = nltk.word_tokenize(sentence.lower())
        seqs = []
        for word in words:
            if word in word2index:
                seqs.append(word2index[word])
            else:
                seqs.append(word2index["UNK"])
        X[i] = seqs
        y[i] = int(label)
        i += 1

# Pad the sequences (left padded with zeros)
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

# Split input into training and test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2,
                                                random_state=42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

42
2328
(5668, 40) (1418, 40) (5668,) (1418,)


In [4]:
# Build model
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE,
                    input_length=MAX_SENTENCE_LENGTH))
model.add(Dropout(0.5))
model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.summary()

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 128)           256256    
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 305,729
Trainable params: 305,729
Non-trainable params: 0
_________________________________________________________________


In [5]:
history = model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    callbacks=[TensorBoard(LOG_DIR)],
                    validation_data=(Xtest, ytest))

Train on 5668 samples, validate on 1418 samples
Epoch 1/10
5668/5668 [==============================] - 27s 5ms/step - loss: 0.3090 - acc: 0.8615 - val_loss: 0.0891 - val_acc: 0.9669
Epoch 2/10
5668/5668 [==============================] - 25s 4ms/step - loss: 0.0515 - acc: 0.9832 - val_loss: 0.0562 - val_acc: 0.9788
Epoch 3/10
5668/5668 [==============================] - 25s 4ms/step - loss: 0.0249 - acc: 0.9908 - val_loss: 0.0455 - val_acc: 0.9831
Epoch 4/10
5668/5668 [==============================] - 25s 4ms/step - loss: 0.0200 - acc: 0.9944 - val_loss: 0.0458 - val_acc: 0.9810
Epoch 5/10
5668/5668 [==============================] - 26s 5ms/step - loss: 0.0103 - acc: 0.9968 - val_loss: 0.0442 - val_acc: 0.9859
Epoch 6/10
5668/5668 [==============================] - 25s 4ms/step - loss: 0.0092 - acc: 0.9982 - val_loss: 0.0436 - val_acc: 0.9859
Epoch 7/10
5668/5668 [==============================] - 26s 4ms/step - loss: 0.0049 - acc: 0.9988 - val_loss: 0.0485 - val_acc: 0.9887
Epoch 8

In [6]:
# evaluate
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score, acc))

for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1, 40)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0].tolist() if x != 0])
    print("{:.0f}\t{:.0f}\t{}".format(ypred, ylabel, sent))

1418/1418 [==============================] - 2s 1ms/step
Test score: 0.043, accuracy: 0.990
1	1	and mission impossible orig tv shows are really awesome ... ... .
0	0	everyone knows brokeback mountain is going to win all because of the stupid gay cowboys .
0	0	da vinci code sucks .
1	1	the people who are worth it know how much i love the da vinci code .
0	0	by the way , the da vinci code sucked , just letting you know ...
